<a href="https://colab.research.google.com/github/Kavyasri275/Appian-AI-Compliance-Assistant/blob/main/Appian_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 1. Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama server in the background
import subprocess
import threading
import time

def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5) # Wait for it to start

!ollama pull qwen2.5:7b       # This is your "Brain"
!ollama pull nomic-embed-text

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.




In [18]:
!nvidia-smi

Thu Dec 25 17:00:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
!pip install -q langchain-ollama langchain-community faiss-cpu pypdf

In [5]:
!pip install -q langchain-text-splitters

In [7]:
!pip install -q langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 33.7 MB/s eta 0:00:00


In [20]:
import os
# --- 1. STABLE CLASSIC IMPORTS ---
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA  # Using the classic chain you installed
from langchain_core.prompts import PromptTemplate

In [21]:
# --- 2. KNOWLEDGE BASE BUILDER ---
def build_unified_knowledge_base(folder_path="./knowledge_folder/"):
    # Ensure folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"FOLDER CREATED: Please upload your PDFs to {folder_path}")
        return None

    print(f"Loading documents from {folder_path}...")

    # Use DirectoryLoader to unify all PDFs
    # NOTE: In Colab, you may need: !pip install unstructured
    loader = DirectoryLoader(folder_path, glob="./*.pdf", loader_cls=PyPDFLoader)
    data = loader.load()

    if not data:
        print("Error: No PDF files found! Upload policy.pdf and SOP_Florida.pdf.")
        return None

    # Text Splitting
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    chunks = text_splitter.split_documents(data)

    # Embeddings and Vector DB
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    vector_db = FAISS.from_documents(chunks, embeddings)
    return vector_db

# --- 3. THE AI AGENT LOGIC ---
def ask_enterprise_ai(question, vector_db):
    # Initialize Qwen 2.5
    llm = ChatOllama(model="qwen2.5:7b", temperature=0)

    # Define the template (Corrected Indentation)
    template = """You are a 'Just-in-Time' Knowledge Assistant for Appian agents.

Your answer MUST be based ONLY on the provided context.
1. If the information is not in the context, clearly state: "I cannot find this rule in the current policy; please consult a human supervisor."
2. Do not use outside knowledge.
3. If you find a rule, cite the file name.

Context: {context}
Question: {question}

Helpful Answer:"""

    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # Building the Chain (Using search_kwargs={"k": 6} to ensure it reads both PDFs)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 6}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    return qa_chain.invoke({"query": question})



In [22]:
folder = "./knowledge_folder/"
db = build_unified_knowledge_base(folder)

if db:
    print("\n--- PROCESSING AGENT QUERY ---")
    query = "What is the flood deductible in Florida and the filing deadline?"
    result = ask_enterprise_ai(query, db)

    # In Classic RetrievalQA, the keys are "result" and "source_documents"
    print("\n" + "="*40)
    print("AI SUGGESTION:", result["result"])
    print("="*40)

    print("\n--- VERIFIABLE PROVENANCE (Audit Trail) ---")
    # This loop proves the data is NOT fragmented anymore
    for doc in result["source_documents"]:
        print(f"✔ Ref: {doc.metadata['source']} | Page: {doc.metadata.get('page', 0)}")

Loading documents from ./knowledge_folder/...

--- PROCESSING AGENT QUERY ---

AI SUGGESTION: The standard $2,500 deductible applies to all flood claims under $50,000. Residential flood claims must be filed within 48 hours of the weather event's conclusion to be eligible for "Fast-Track" processing according to Florida Regional Operating Procedure (SOP) v2025.4.

--- VERIFIABLE PROVENANCE (Audit Trail) ---
✔ Ref: knowledge_folder/SOP_Florida.pdf | Page: 0
✔ Ref: knowledge_folder/policy.pdf | Page: 0
✔ Ref: knowledge_folder/policy.pdf | Page: 0
✔ Ref: knowledge_folder/policy.pdf | Page: 0


In [23]:
# --- FINAL TEST FOR PPT SCREENSHOT ---
test_claim = "Is a pipe burst in Florida considered a flood claim?"

print(f"Testing Claim: {test_claim}")
res = ask_enterprise_ai(test_claim, db)

# Use ["result"] for the answer
print(f"\nAI GUIDANCE: {res['result']}")

# Use ["source_documents"] to get the metadata
# We take the first document [0] to show the specific page
source_file = res['source_documents'][0].metadata['source']
page_num = res['source_documents'][0].metadata['page']

print(f"SOURCE: {source_file} (Page {page_num})")

Testing Claim: Is a pipe burst in Florida considered a flood claim?

AI GUIDANCE: No, according to Rule FL-101 in the ENTERPRISE INSURANCE & COMPLIANCE STANDARD (v2025.1), damage caused by internal pipe bursts is NOT categorized as "Flood" and must be filed under "Maintenance & Repair."
SOURCE: knowledge_folder/policy.pdf (Page 0)


In [24]:
# --- EDGE CASE TEST: A TOPIC NOT IN YOUR PDF ---
edge_query = "Is cosmetic surgery covered under the Florida flood policy?"

print(f"Agent Inquiry: {edge_query}")
result = ask_enterprise_ai(edge_query, db)

print("\n--- SAFETY CHECK OUTPUT ---")
print(result["result"])

Agent Inquiry: Is cosmetic surgery covered under the Florida flood policy?

--- SAFETY CHECK OUTPUT ---
I cannot find this rule in the current policy; please consult a human supervisor.


In [25]:
import gradio as gr
import time

# --- 1. THE LOGIC FUNCTION (Updated for 'messages' format) ---
def chat_with_assistant(message, history):
    if 'db' not in globals() or db is None:
        error_msg = {"role": "assistant", "content": "⚠️ Error: Knowledge base not initialized. Please run the setup cell first."}
        return "", history + [error_msg]

    # Run your existing RAG logic
    response = ask_enterprise_ai(message, db)

    # Extract the answer
    answer = response["result"]

    # Build a professional Provenance section
    provenance = "\n\n---\n### 🔍 VERIFIABLE PROVENANCE\n"
    unique_sources = []
    for doc in response["source_documents"]:
        source_label = f"📄 **{doc.metadata['source'].split('/')[-1]}** (Page {doc.metadata.get('page', 0)})"
        if source_label not in unique_sources:
            unique_sources.append(source_label)
            provenance += f"- {source_label}\n"

    full_response = f"{answer}{provenance}"

    # Update history using the new 'messages' format
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": full_response})

    return "", history

# --- 2. THE UI DESIGN ---
# FIX: 'theme' goes here in the constructor, NOT in launch()
with gr.Blocks(theme=gr.themes.Soft(), title="Appian AI Copilot") as demo:
    gr.Markdown("# 🛡️ Appian Just-in-Time Knowledge Assistant")
    gr.Markdown("*Proactive compliance and policy retrieval for high-stakes insurance casework.*")

    with gr.Row():
        # Sidebar-style Column
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### ⚙️ System Status")
            gr.Markdown("✅ **LLM:** Qwen 2.5 (Local)")
            gr.Markdown("✅ **Database:** FAISS Vector Store")
            gr.Markdown("✅ **Privacy:** 100% On-Premise")

            gr.HTML("<hr>")
            gr.Markdown("### 💡 Sample Inquiries")

            # Pre-defining examples for the helper below
            example_list = [
                ["What is the Florida flood deductible?"],
                ["Is a pipe burst covered under flood policy?"],
                ["What is the filing deadline for Florida claims?"]
            ]

        # Chat Column
        with gr.Column(scale=4):
            # FIX: Added type="messages" to match the logic above
            chatbot = gr.Chatbot(
                label="Case Assistant Stream",
                type="messages",
                height=500
            )

            with gr.Row():
                msg = gr.Textbox(
                    show_label=False,
                    placeholder="Analyze active case data (e.g., Claim Type: Flood, State: Florida)...",
                    container=False,
                    scale=7
                )
                submit = gr.Button("Analyze Case", variant="primary", scale=1)

            clear = gr.ClearButton([msg, chatbot], value="Clear Case")

            # Link examples to the msg textbox
            gr.Examples(examples=example_list, inputs=[msg])

    # Wire up the logic
    msg.submit(chat_with_assistant, [msg, chatbot], [msg, chatbot])
    submit.click(chat_with_assistant, [msg, chatbot], [msg, chatbot])

# Launching without the 'theme' argument to avoid TypeError
demo.launch(share=True, debug=True)

/tmp/ipython-input-4193406589.py:35: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Appian AI Copilot") as demo:
/tmp/ipython-input-4193406589.py:60: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://32e57a227cec8995ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://32e57a227cec8995ae.gradio.live
